# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import geoapify_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Khatanga,71.9667,102.5000,-34.85,99,55,6.13,RU,1672119233
1,1,Flinders,-34.5833,150.8552,84.18,67,3,8.99,AU,1672119234
2,2,Harper,4.3750,-7.7169,76.33,82,31,6.96,LR,1672119234
3,3,Busselton,-33.6500,115.3333,75.58,36,2,16.02,AU,1672119235
4,4,Qaanaaq,77.4840,-69.3632,-10.62,99,100,5.66,GL,1672119235


---

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.


In [21]:
# Configure gmaps
gmaps.configure(api_key=geoapify_key)

# Lat and Lng as locations and Humidity as weight
locations = city_data_df[['Lat', 'Lng']]
weight = city_data_df['Humidity'].astype(float)
humidity = city_data_df['Humidity'].max()

In [22]:
# Plot Heatmap
heatmap = gmaps.figure()

# Create Heatmap layer
heatmap_layer_ = gmaps.heatmap_layer(locations, weights = weight, dissipating = False, max_intensity=humidity, point_radius=1)

# Add Heatmap layer
heatmap.add_layer(heatmap_layer_)

# Show Heatmap
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# New DF to fit weather conditions
weather_df = city_data_df.loc[(((city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80)) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] >= 0)), :]

# Drop any rows with null values
weather_df = city_data_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Khatanga,71.9667,102.5000,-34.85,99,55,6.13,RU,1672119233
1,1,Flinders,-34.5833,150.8552,84.18,67,3,8.99,AU,1672119234
2,2,Harper,4.3750,-7.7169,76.33,82,31,6.96,LR,1672119234
3,3,Busselton,-33.6500,115.3333,75.58,36,2,16.02,AU,1672119235
4,4,Qaanaaq,77.4840,-69.3632,-10.62,99,100,5.66,GL,1672119235
...,...,...,...,...,...,...,...,...,...,...
577,577,Upington,-28.4478,21.2561,71.98,33,0,3.44,ZA,1672119593
578,578,Vanimo,-2.6741,141.3028,80.89,85,100,7.25,PG,1672119593
579,579,Tainan City,22.9908,120.2133,76.78,33,20,3.44,TW,1672119594
580,580,Linxia Chengguanzhen,35.6003,103.2064,25.93,34,95,3.27,CN,1672119594


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(25)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Khatanga,RU,71.9667,102.5000,99,
1,Flinders,AU,-34.5833,150.8552,67,
2,Harper,LR,4.3750,-7.7169,82,
3,Busselton,AU,-33.6500,115.3333,36,
4,Qaanaaq,GL,77.4840,-69.3632,99,
5,Mahébourg,MU,-20.4081,57.7000,83,
6,Lebu,CL,-37.6167,-73.6500,86,
7,Kolondiéba,ML,11.0882,-6.8926,28,
8,Bluff,NZ,-46.6000,168.3333,71,
9,Albany,US,42.6001,-73.9662,65,


In [26]:
# Set parameters to search for a hotel
params = {
   'radius': 5000, 
   'types': 'hotel',
   'keyword': 'hotel',
   'key': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{lat},{lng}"
    # params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
        
# Display sample data
hotel_df

Starting hotel search
Khatanga - nearest hotel: 
Flinders - nearest hotel: 
Harper - nearest hotel: 
Busselton - nearest hotel: 
Qaanaaq - nearest hotel: 
Mahébourg - nearest hotel: 
Lebu - nearest hotel: 
Kolondiéba - nearest hotel: 
Bluff - nearest hotel: 
Albany - nearest hotel: 
Taft - nearest hotel: 
Kaitangata - nearest hotel: 
Krasnosel'kup - nearest hotel: 
Bermeo - nearest hotel: 
Alyangula - nearest hotel: 
Umm Kaddadah - nearest hotel: 
Cabo San Lucas - nearest hotel: 
Magadan - nearest hotel: 
Port Blair - nearest hotel: 
Rikitea - nearest hotel: 
Poum - nearest hotel: 
Tromsø - nearest hotel: 
Hermanus - nearest hotel: 
Lufilufi - nearest hotel: 
Cape Town - nearest hotel: 
Vaini - nearest hotel: 
Ushuaia - nearest hotel: 
Punta Arenas - nearest hotel: 
Kahului - nearest hotel: 
Dikson - nearest hotel: 
Goundam - nearest hotel: 
Padang - nearest hotel: 
Cabedelo - nearest hotel: 
Kerrobert - nearest hotel: 
Karratha - nearest hotel: 
Varhaug - nearest hotel: 
Cidreira - ne

KeyboardInterrupt: 

In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Configure the map plot
hotels = hotel_df[['City', 'Country', 'Hotel Name']]
markers = gmaps.marker_layer(locations)
heatmap.add_layer(markers)

# Display the map
heatmap

Figure(layout=FigureLayout(height='420px'))